# 라벨 전처리 코드
라벨이 영상마다의 json파일로 흩어져있어, 훈련에 바로 사용하거나 데이터분석을 하기에 어려움이 있었음.  
이에 모든 json을 key-value를 기준으로 하나의 csv파일로 통합하는 과정 진행.

# 1. 환경 설정

In [1]:
import pandas as pd
import glob
import json
import os

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# 2. json파일들을 수합하여 데이터프레임으로 만드는 함수 선언

In [3]:
def json_to_csv(jsonpath):
    
    # 원본 라벨에는 bb_1로 시작하는 파일명도 존재. bb_1은 1인칭, bb_3은 3인칭. 본 모델은 3인칭 영상만 활용하므로 제목 한정
    json_files = glob.glob(f"{jsonpath}/bb_3*.json") 

    # json의 모든 key들을 가져와서, 앞으로 생성할 데이터프레임의 column명으로 정리
    with open(json_files[0],'r') as f:
        temp_json = json.load(f)
    json_dict = {key:[] for key in temp_json['video'].keys()}

    for iter in range(len(json_files)):
        with open(json_files[iter],'r') as f:
            temp_json = json.load(f)
    for key in temp_json.keys():
        if key not in json_dict.keys():
            json_dict[key] = []  # 새로운 키일 경우 값 할당
            
    # json의 모든 value를 가져와서 기입
    for file in json_files:
        with open(file, 'r') as f:
            json_data = json.load(f)
            for key in json_dict.keys():
              try:
                json_dict[key].append(json_data['video'][key])
              except:
                json_dict[key].append(None)   
                
    return pd.DataFrame(json_dict)

# 3. 만든 데이터프레임을 하나의 데이터프레임으로 수합

In [4]:
train_path = './files/샘플데이터셋/train/json'
val_path = './files/샘플데이터셋/validation/json'

train_df = json_to_csv(train_path)
val_df = json_to_csv(val_path)

In [5]:
# 모든 행의 이름이 일치하므로 두 데이터프레임을 세로로 합칠 수 있다.
train_df.columns == val_df.columns 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [6]:
#train_df.columns == val_df.columns # 모든 행의 이름이 일치하므로 두 데이터프레임을 세로로 합칠 수 있다.

# 수합 후에도 훈련 셋과 검증 셋을 구분하기 위해 정보컬럼 추가
train_df['데이터유형'] = 'train'
val_df['데이터유형'] = 'val'
whole_df = pd.concat([train_df, val_df])
whole_df

,video_name,video_date,filming_way,video_point_of_view,accident_negligence_rateA,accident_negligence_rateB,accident_object,accident_place,accident_place_feature,vehicle_a_progress_info,...,damage_location,related_laws,violation_of_law,additional_elements,violation_factor,weather,traffic_accident_type,info,video,데이터유형
0,bb_3_120224_vehicle_244_47620,120417,bb,3,10,90,0,3,11,25,...,,,,,,,110,None,None,train
1,bb_3_130314_vehicle_126_251,130314,bb,3,30,70,0,0,6,9,...,,,,,,,11,None,None,train
2,bb_3_131015_vehicle_116_150,131015,bb,3,100,0,0,13,41,91,...,,,,,,,194,None,None,train
3,bb_3_180126_vehicle_206_34957,180126,bb,3,0,100,0,2,18,47,...,,,,,,,60,None,None,train
4,bb_3_210902_two-wheeled-vehicle_314_014,210902,bb,3,35,65,2,1,10,104,...,,,,,,,226,None,None,train
0,bb_3_130224_vehicle_255_26330,121008,bb,3,0,100,0,0,0,2,...,,,,,,,2,None,None,val
1,bb_3_140207_vehicle_46_076,140207,bb,3,30,70,0,1,11,25,...,,,,,,,31,None,None,val
2,bb_3_141102_vehicle_218_22430,141102,bb,3,100,0,0,13,41,91,...,,,,,,,194,None,None,val
3,bb_3_150108_two-wheeled-vehicle_234_21105,150817,bb,3,20,80,2,3,10,107,...,,,,,,,337,None,None,val
4,bb_3_160528_vehicle_202_35512,160528,bb,3,80,20,0,2,59,174,...,,,,,,,79,None,None,val


# 4. 라벨 정의표를 기준으로 라벨을 식별 가능한 정보로 변환
해당 json파일들의 value들은 숫자로 인코딩되어있어 의미를 식별하기 어려웠음.  
모델링과 데이터 분석의 용이함을 위한 인코딩 전 정보가 담긴 라벨 정의표를 가져와서 정보를 변환.  
사고 유형과 traffic_accident_type 컬럼을 기준으로 merge하여 이를 수행.

In [7]:
# whole_df가 생성되었으니 라벨 정의표를 가져와 사고 유형을 기준으로 merge하면, 학습에 활용할 라벨 csv파일을 최종 생성할 수 있음
basic_table = pd.read_csv('./files/라벨정의표.csv', encoding = 'cp949')

In [8]:
merged_df = whole_df.merge(basic_table, how='inner', left_on='traffic_accident_type', right_on='사고유형')

In [9]:
# 비디오 고유정보와 사고타입번호만 남기고 한글 정보를 가져옴
json_final = merged_df[['video_name','video_date','사고객체','사고장소','사고장소특징','A진행방향','B진행방향','과실비율A','과실비율B','사고유형','데이터유형']].rename({'video_name':'영상파일명','video_date':'촬영일자'}, axis=1)

In [10]:
# 확장자명을 파일명에 추가
json_final['영상파일명'] = [i+'.mp4' for i in list(json_final['영상파일명'])]

In [12]:
# 모델에서 활용하기 위한 최종 json 파일을 저장 
json_final.to_csv('./files/데이터라벨.csv', encoding = 'cp949', index  = False)